In [ ]:
# from pythainlp import word_tokenize
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
# from pythainlp.word_vector import thai2vec 
import numpy as np
import pandas as pd
%reload_ext autoreload
%autoreload 2
# %matplotlib inline

from pythainlp.tokenize import word_tokenize,sent_tokenize
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from scipy.cluster import hierarchy
import dill as pickle
import pandas as pd
import pythainlp.word_vector
# model = word_vector.get_model()
model_path = 'thwiki_data/models/'

In [ ]:
df = pd.read_csv("data/dataReV5.csv")
df

In [ ]:
text = df['Problem'].dropna()
text

In [ ]:
from pythainlp import word_tokenize, Tokenizer

In [ ]:
sentences_newmm= []
sentences_longest= []

for x in text:
    proc_newmm = word_tokenize(x, engine='newmm', keep_whitespace=False)
    proc_longest = word_tokenize(x, engine='longest', keep_whitespace=False)

    lst_newmm = []
    lst_longest = []
    
    for word in proc_newmm:
        if(word != " "):
            lst_newmm.append(word)
    sentences_newmm.append(lst_newmm)
    for word in proc_longest:
        if(word != " "):
            lst_longest.append(word)
    sentences_longest.append(lst_longest)

In [ ]:
model_nlp = pythainlp.word_vector.get_model()

In [ ]:
def sentence_vectorizer(ss,dim=300,use_mean=True): # ประกาศฟังก์ชัน sentence_vectorizer
    s = word_tokenize(ss)
    vec = np.zeros((1,dim))
    for word in s:
        if word in model_nlp.wv.index2word: 
            vec+= model_nlp.wv.word_vec(word)
        else: pass
    if use_mean: vec /= len(s)
    return vec

lst_vector = []
for i in text:
    lst_vector.append(sentence_vectorizer(i)[0])
X=np.array(lst_vector)
print(X)

In [ ]:
import matplotlib.pyplot as plt
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

In [ ]:
elbow=elbow_point(wcss)
elbow.dtype

In [ ]:
n = len(wcss)+1
K= range(1, n)

print("Optimal Cluster Number: ",K[elbow])
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(K, wcss, 'bx-')
ax.plot(K[elbow], wcss[elbow], marker='o', markersize=12,markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
# ax.set_xticks(np.arange(0,n,1))
# ax.set_yticks(np.arange(0,max(wcss),0.5))
plt.grid()
plt.xlabel('Number of clusters')
plt.ylabel('Average within-cluster sum of squares')
plt.title('Elow for KMeans clustering')
plt.show();

In [ ]:
n_clusters = K[elbow]
clf = KMeans(n_clusters=n_clusters,
            max_iter=100,
            init='k-means++',
            n_init=1)
labels = clf.fit_predict(X)
labels

In [ ]:
n_clusters = 5
clf = KMeans(n_clusters=n_clusters,
            max_iter=100,
            init='k-means++',
            n_init=1)
labels = clf.fit_predict(X)
labels
cluster = []
value = []
o = ""
for index, sentence in enumerate(sentences_newmm):
#     print(str(labels[index]) + ":" + str(sentence))
    cluster.append(labels[index])
    value.append(o.join(sentences_newmm[index]))
list_of_cluster = list(zip(cluster,value))

In [ ]:
df_cluster = pd.DataFrame(list_of_cluster, columns = ['cluster', 'text'])  
df_cluster